In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent.parent)) ## ../../covid19datascience* <- set parent level here

In [ ]:
import pandas as pd
from epiweeks import Week
from datetime import timedelta, date
from os.path import join
from unidecode import unidecode

from bulletin import default_input, default_output, root, hoje, data_comeco_pandemia
from bulletin.utils.normalize import normalize_text, normalize_hash
from bulletin.systems.sim import Sim
from bulletin.systems.sivep import Sivep
from bulletin.systems.notifica import Notifica
from bulletin.systems.casos_confirmados import CasosConfirmados

In [ ]:
import os
import re
import random
import time
import binascii
from bisect import bisect_right
from heapq import heappop, heappush

In [ ]:
from tqdm import tqdm

In [ ]:
def match_plagiaries(dict, df):
    # Map the two documents to each other.
    dict[df.index.values[0]] = df['id_duplicado'].values[0]
    dict[df['id_duplicado'].values[0]] = df.index.values[0]

In [ ]:
cc = CasosConfirmados()
cc.load('cc_19_08_2021')
covid_obitos = cc.df.loc[cc.df['evolucao']==2].copy()

In [ ]:
del cc

In [ ]:
# sim = Sim()
# sim.load('dopr')
#sim.read_all_database_files()
#sim.fix_dtypes()
#sim.save(replace=True)

In [ ]:
# sim.df = sim.df.loc[
#     sim.df['CAUSABAS'].str.contains('B342') |
#     sim.df['LINHAA'].str.contains('B342') |
#     sim.df['LINHAB'].str.contains('B342') |
#     sim.df['LINHAC'].str.contains('B342') |
#     sim.df['LINHAC'].str.contains('B342') |
#     sim.df['LINHAII'].str.contains('B342')
# ]

In [ ]:
# sim.to_notifica()
# sim.hashes()

In [ ]:
#sim_covid_obitos = sim.df

In [ ]:
#sim_covid_obitos = sim_covid_obitos.loc[sim_covid_obitos['ibge_residencia']=='410690'].copy()
#sim_covid_obitos.shape[0]

In [ ]:
#del sim

In [ ]:
# sim_covid_obitos = sim_covid_obitos.drop_duplicates('id')

In [ ]:
# for ident in sim_covid_obitos.loc[sim_covid_obitos['hash_mae'].notna() & sim_covid_obitos.duplicated('hash_mae',keep='first')]['id'].values:
#     sim_covid_obitos.loc[sim_covid_obitos['id']==ident, 'id_duplicado'] = sim_covid_obitos.loc[(sim_covid_obitos['hash_mae']==sim_covid_obitos.loc[sim_covid_obitos['id']==ident]['hash_mae'].values[0]) & (sim_covid_obitos.duplicated('hash_mae',keep='last'))]['id'].values[0]

In [ ]:
# sim_covid_obitos.loc[sim_covid_obitos['id_duplicado'].notna()].shape[0]

In [ ]:
# sivep = Sivep()
# sivep.load('sraghospitalizado')
# sivep.read_all_database_files()
# sivep.fix_dtypes()
# sivep.save(replace=True)

In [ ]:
# sivep.to_notifica()
# sivep.hashes()

In [ ]:
# sivep_covid_obitos = sivep.get_obitos()

In [ ]:
# sivep_covid_obitos = sivep_covid_obitos.loc[sivep_covid_obitos['ibge_residencia']=='411520.0']
# sivep_covid_obitos.shape[0]

In [ ]:
for ident in covid_obitos.loc[covid_obitos['hash_mae'].notna() & covid_obitos.duplicated('hash_mae',keep='first')]['id'].values:
    covid_obitos.loc[covid_obitos['id']==ident, 'id_duplicado'] = covid_obitos.loc[(covid_obitos['hash_mae']==covid_obitos.loc[covid_obitos['id']==ident]['hash_mae'].values[0]) & (covid_obitos.duplicated('hash_mae',keep='last'))]['id'].values[0]

In [ ]:
covid_obitos.loc[covid_obitos['id_duplicado'].notna()].shape[0]

In [ ]:
# for ident in sivep_covid_obitos.loc[sivep_covid_obitos['hash_mae'].notna() & sivep_covid_obitos.duplicated('hash_mae',keep='first')]['id'].values:
#     sivep_covid_obitos.loc[sivep_covid_obitos['id']==ident, 'id_duplicado'] = sivep_covid_obitos.loc[(sivep_covid_obitos['hash_mae']==sivep_covid_obitos.loc[sivep_covid_obitos['id']==ident]['hash_mae'].values[0]) & (sivep_covid_obitos.duplicated('hash_mae',keep='last'))]['id'].values[0]

In [ ]:
# sivep_covid_obitos.loc[sivep_covid_obitos['id_duplicado'].notna()].shape[0]

In [ ]:
# del sivep

In [ ]:
# interseccao_hash = sim_covid.loc[
#     sim_covid['hash_mae'].notna() & sim_covid['hash_mae'].isin(sivep_covid['hash_mae'])
# ].copy()

In [ ]:
# print(f"Óbitos confirmados SIVEP: {sivep_covid.shape[0]}")
# print(f"Óbitos com menção ao covid SIM: {sim_covid.shape[0]}")
# print(f"Intersecção das duas bases de dados usando hash: {interseccao_hash.shape[0]}")

In [ ]:
# dataFile = sim_covid.loc[(~sim_covid['hash_mae'].isnull())].copy()
# truthFile = sivep_covid.loc[(~sivep_covid['hash_mae'].isnull())].drop_duplicates('hash_mae', keep = 'last').copy()

In [ ]:
#truthFile.loc[truthFile['hash_mae'].isin(dataFile['hash_mae']), 'id_in_sim'] = dataFile.loc[dataFile['hash_mae'].isin(truthFile['hash_mae']), 'id']
#del truthFile['id_in_sim']
# for mae in truthFile['hash_mae'].values:
#     if (mae in dataFile['hash_mae'].values):
#         truthFile.loc[truthFile['hash_mae']==mae, 'id_in_sim'] = dataFile.loc[dataFile['hash_mae']==mae]['id'].values[0]

In [ ]:
dataFile = covid_obitos.loc[covid_obitos['hash_mae'].notna()]
truthFile = covid_obitos

In [ ]:
# dataFile = sivep_covid_obitos.loc[sivep_covid_obitos['hash_mae'].notna()]
# truthFile = sivep_covid_obitos

In [ ]:
# dataFile = sim_covid_obitos.loc[sim_covid_obitos['hash_mae'].notna()]
# truthFile = sim_covid_obitos

In [ ]:
dataFile = dataFile.set_index('id')
truthFile = truthFile.set_index('id')

In [ ]:
# del sim_covid
# del sivep_covid

In [ ]:
numHashes = 10
numDocs = dataFile.shape[0]# if (dataFile.shape[0] > truthFile.shape[0]) else truthFile.shape[0]
print([numDocs, numHashes])

In [ ]:
#truthFile.loc[truthFile['id_in_sim'].notna()]
#truthFile.loc[truthFile.index=='315849756828']
#dataFile.loc[dataFile.index=='285711644']

In [ ]:
# =============================================================================
#                  Parse The Ground Truth Tables
# =============================================================================
# Build a dictionary mapping the document IDs to their plagiaries, and vice-
# versa.
plagiaries = {}

for idx in truthFile.loc[truthFile['id_duplicado'].notna()].index:
    match_plagiaries(plagiaries,truthFile.loc[truthFile.index==idx])
plagiaries

In [ ]:
# =============================================================================
#               Convert Documents To Sets of Shingles
# =============================================================================

# The current shingle ID value to assign to the next new shingle we 
# encounter. When a shingle gets added to the dictionary, we'll increment this
# value.
curShingleID = 0

# Create a dictionary of the articles, mapping the article identifier (e.g., 
# "t8470") to the list of shingle IDs that appear in the document.
docsAsShingleSets = {}

docNames = []

t0 = time.time()

totalShingles = 0

k = range(3)

for i in tqdm(dataFile.index):
  
  # Retrieve the article ID, which is the first word on the line.  
  docID = i
  
  # Maintain a list of all document IDs.  
  docNames.append(docID)
  
  # 'shinglesInDoc' will hold all of the unique shingle IDs present in the 
  # current document. If a shingle ID occurs multiple times in the document,
  # it will only appear once in the set (this is a property of Python sets).
  shinglesInDoc = set()
  
  # For each word in the document...
  for index in range(0, len(dataFile['hash_mae'][i]) - 2):
    # Construct the shingle text by combining three words together.
    shingle = ''
    for j in k:
      shingle = shingle+dataFile['hash_mae'][i][index+j]

    # Hash the shingle to a 32-bit integer.
    crc = binascii.crc32(bytearray(shingle, "utf8")) & 0xffffffff
    
    # Add the hash value to the list of shingles for the current document. 
    # Note that set objects will only add the value to the set if the set 
    # doesn't already contain it. 
    shinglesInDoc.add(crc)
  
  # Store the completed list of shingles for this document in the dictionary.
  docsAsShingleSets[docID] = shinglesInDoc
  
  # Count the number of shingles across all documents.
  totalShingles = totalShingles + (len(dataFile['hash_mae'][i]) - 2)

# Report how long shingling took.
print('\nShingling ' + str(numDocs) + ' docs took %.2f sec.' % (time.time() - t0))
 
print('\nAverage shingles per doc: %.2f' % (totalShingles / numDocs))

In [ ]:
# =============================================================================
#                     Define Triangle Matrices
# =============================================================================

# Define virtual Triangle matrices to hold the similarity values. For storing
# similarities between pairs, we only need roughly half the elements of a full
# matrix. Using a triangle matrix requires less than half the memory of a full
# matrix, and can protect the programmer from inadvertently accessing one of
# the empty/invalid cells of a full matrix.

# Calculate the number of elements needed in our triangle matrix
numElems = int(numDocs * (numDocs - 1) / 2)

# Initialize two empty lists to store the similarity values. 
# 'JSim' will be for the actual Jaccard Similarity values. 
# 'estJSim' will be for the estimated Jaccard Similarities found by comparing
# the MinHash signatures.
JSim = [0 for x in range(numElems)]
estJSim = [0 for x in range(numElems)]

# Define a function to map a 2D matrix coordinate into a 1D index.
def getTriangleIndex(i, j):
  # If i == j that's an error.
  if i == j:
    sys.stderr.write("Can't access triangle matrix with i == j")
    sys.exit(1)
  # If j < i just swap the values.
  if j < i:
    temp = i
    i = j
    j = temp
  
  # Calculate the index within the triangular array.
  # This fancy indexing scheme is taken from pg. 211 of:
  # http://infolab.stanford.edu/~ullman/mmds/ch6.pdf
  # But I adapted it for a 0-based index.
  # Note: The division by two should not truncate, it
  #       needs to be a float. 
  k = int(i * (numDocs - (i + 1) / 2.0) + j - i) - 1
  
  return k

In [ ]:
# =============================================================================
#                 Generate MinHash Signatures
# =============================================================================

# Time this step.
t0 = time.time()

print('\nGenerating random hash functions...')

# Record the maximum shingle ID that we assigned.
maxShingleID = 2**32-1

# We need the next largest prime number above 'maxShingleID'.
# I looked this value up here: 
# http://compoasso.free.fr/primelistweb/page/prime/liste_online_en.php
nextPrime = 4294967311


# Our random hash function will take the form of:
#   h(x) = (a*x + b) % c
# Where 'x' is the input value, 'a' and 'b' are random coefficients, and 'c' is
# a prime number just greater than maxShingleID.

# Generate a list of 'k' random coefficients for the random hash functions,
# while ensuring that the same value does not appear multiple times in the 
# list.
def pickRandomCoeffs(k):
  # Create a list of 'k' random values.
  randList = []
  
  while k > 0:
    # Get a random shingle ID.
    randIndex = random.randint(0, maxShingleID) 
  
    # Ensure that each random number is unique.
    while randIndex in randList:
      randIndex = random.randint(0, maxShingleID) 
    
    # Add the random number to the list.
    randList.append(randIndex)
    k = k - 1
    
  return randList

# For each of the 'numHashes' hash functions, generate a different coefficient 'a' and 'b'.   
coeffA = pickRandomCoeffs(numHashes)
coeffB = pickRandomCoeffs(numHashes)

print('\nGenerating MinHash signatures for all documents...')

# List of documents represented as signature vectors
signatures = []

# Rather than generating a random permutation of all possible shingles, 
# we'll just hash the IDs of the shingles that are *actually in the document*,
# then take the lowest resulting hash code value. This corresponds to the index 
# of the first shingle that you would have encountered in the random order.

# For each document...
for docID in docNames:
  
  # Get the shingle set for this document.
  shingleIDSet = docsAsShingleSets[docID]
  
  # The resulting minhash signature for this document. 
  signature = []
  
  # For each of the random hash functions...
  for i in range(0, numHashes):
    
    # For each of the shingles actually in the document, calculate its hash code
    # using hash function 'i'. 
    
    # Track the lowest hash ID seen. Initialize 'minHashCode' to be greater than
    # the maximum possible value output by the hash.
    minHashCode = nextPrime + 1
    
    # For each shingle in the document...
    for shingleID in shingleIDSet:
      # Evaluate the hash function.
      hashCode = (coeffA[i] * shingleID + coeffB[i]) % nextPrime 
      
      # Track the lowest hash code seen.
      if hashCode < minHashCode:
        minHashCode = hashCode

    # Add the smallest hash code value as component number 'i' of the signature.
    signature.append(minHashCode)
  
  # Store the MinHash signature for this document.
  signatures.append(signature)

# Calculate the elapsed time (in seconds)
elapsed = (time.time() - t0)
        
print("\nGenerating MinHash signatures took %.2fsec" % elapsed)

In [ ]:
# =============================================================================
#                 Calculate Jaccard Similarities
# =============================================================================
# In this section, we will directly calculate the Jaccard similarities by 
# comparing the sets. This is included here to show how much slower it is than
# the MinHash approach.

# Calculating the Jaccard similarities gets really slow for large numbers
# of documents.
if numDocs <= 2500:
#if True:
    print("\nCalculating Jaccard Similarities...")

    # Time the calculation.
    t0 = time.time()

    # For every document pair...
    for i in range(0, numDocs):
      
      # Print progress every 100 documents.
      if (i % 100) == 0:
        print("  (" + str(i) + " / " + str(numDocs) + ")")

      # Retrieve the set of shingles for document i.
      s1 = docsAsShingleSets[docNames[i]]
      
      for j in range(i + 1, numDocs):
        # Retrieve the set of shingles for document j.
        s2 = docsAsShingleSets[docNames[j]]
        
        # Calculate and store the actual Jaccard similarity.
        JSim[getTriangleIndex(i, j)] = (len(s1.intersection(s2)) / len(s1.union(s2)))    

    # Calculate the elapsed time (in seconds)
    elapsed = (time.time() - t0)
        
    print("\nCalculating all Jaccard Similarities took %.2fsec" % elapsed)

# Delete the Jaccard Similarities, since it's a pretty big matrix.    
del JSim

In [ ]:
# =============================================================================
#                     Compare All Signatures
# =============================================================================  
  
# Creates a N x N matrix initialized to 0.

# Time this step.
t0 = time.time()

# For each of the test documents...
for i in tqdm(range(0, numDocs)):
  # Get the MinHash signature for document i.
  signature1 = signatures[i]
    
  # For each of the other test documents...
  for j in range(i + 1, numDocs):
    
    # Get the MinHash signature for document j.
    signature2 = signatures[j]
    
    count = 0
    # Count the number of positions in the minhash signature which are equal.
    for k in range(0, numHashes):
      count = count + (signature1[k] == signature2[k])
    
    # Record the percentage of positions which matched.    
    estJSim[getTriangleIndex(i, j)] = (count / numHashes)

# Calculate the elapsed time (in seconds)
elapsed = (time.time() - t0)
        
print("\nComparing MinHash signatures took %.2fsec" % elapsed)

In [ ]:
# =============================================================================
#                   Display Similar Document Pairs
# =============================================================================  

# Count the true positives and false positives.
tp = 0
fp = 0
  
threshold = 0.8
print("\nList of Document Pairs with J(d1,d2) more than", threshold)
print("Values shown are the estimated Jaccard similarity and the actual")
print("Jaccard similarity.\n")
print("                   Est. J   Act. J")

# For each of the document pairs...
for i in range(0, numDocs):  
  for j in range(i + 1, numDocs):
    # Retrieve the estimated similarity value for this pair.
    estJ = estJSim[getTriangleIndex(i, j)]
    
    # If the similarity is above the threshold...
    if estJ > threshold:
    
      # Calculate the actual Jaccard similarity for validation.
      s1 = docsAsShingleSets[docNames[i]]
      s2 = docsAsShingleSets[docNames[j]]
      J = 0 if (len(s1.union(s2))==0) else (len(s1.intersection(s2)) / len(s1.union(s2)))
      
      # Print out the match and similarity values with pretty spacing.
      #print("  %5s --> %5s   %.2f     %.2f" % (docNames[i], docNames[j], estJ, J))
      
      # Check whether this is a true positive or false positive.
      # We don't need to worry about counting the same true positive twice
      # because we implemented the for-loops to only compare each pair once.
      try:
          if plagiaries[docNames[i]] == docNames[j]:
              tp = tp + 1
              print("tp:  %5s --> %5s   %.2f     %.2f" % (docNames[i], docNames[j], estJ, J))
              dataFile.loc[[docNames[i]], 'id_duplicado'] = dataFile.loc[[docNames[j]]].index.values[0]
              dataFile.loc[[docNames[j]], 'id_duplicado'] = dataFile.loc[[docNames[i]]].index.values[0]
          else:
              print("fp:  %5s --> %5s   %.2f     %.2f" % (docNames[i], docNames[j], estJ, J))
              dataFile.loc[[docNames[i]], 'id_duplicado'] = dataFile.loc[[docNames[j]]].index.values[0]
              dataFile.loc[[docNames[j]], 'id_duplicado'] = dataFile.loc[[docNames[i]]].index.values[0]
              fp = fp + 1
      except:
          pass


# Display true positive and false positive counts.
print()
print("True positives:  " + str(tp) + " / " + str(int(len(plagiaries.keys()) / 2)))
print("False positives: " + str(fp))

In [ ]:
cc = CasosConfirmados()
cc.df = dataFile
cc.save('obitos_minhash')